# Classification and prediction of blight violations

---

_You are currently looking at **version 1.1** of this notebook._

---

### Detroit blight - Exploring and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant
___


### Import

In [ ]:
% matplotlib inline

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import re

from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
np.set_printoptions(precision=4)

### General Helper functions

In [ ]:
def num_features(df):
    return df.select_dtypes(include=[np.number]).columns.tolist()

def cat_features(df):
    return df.select_dtypes(include=['object', 'category']).columns

def date_features(df):
    return df.columns.str.extractall(r'(.*date.*)')[0].values.tolist()

def clip_outliers(values, p=99):
    """clip at 1.5 IQR"""
    min = np.percentile(values, 100-p)
    max = np.percentile(values, p)
    return np.clip(values, min, max)

def numerize_code(df, feature, replace=0):
    df[feature] = pd.to_numeric(df[feature], errors='coerce')
    nums = df[feature].fillna(replace).astype('int64')
    return clip_outliers(nums)

def alpharize_code(df, feature, bins, replace=0, cut=pd.qcut, upper=True):
    zipcode = numerize_code(df, feature, replace)
    labels = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')[:bins]
    if not upper:
        labels = list(map(lambda x:x.lower(), labels))
    return cut(zipcode, bins, labels=labels)

def factorized(df):
    if isinstance(df, pd.Series):
        return df.factorize()[0]
    return df.loc[:, cat_features(df)].apply(lambda x: x.factorize()[0])

def cat_distribution(df, cat_feature, target):
    df[cat_feature] = df[cat_feature].fillna('missing')
    group = df.groupby([cat_feature, target])[target].count().unstack(level=0).T.fillna(0)
    return group

### Data

In [ ]:
set_types = {'compliance':object}
df_train = pd.read_csv('../_data/train.csv', encoding='latin-1', low_memory=False, dtype=set_types)
df_test = pd.read_csv('../_data/test.csv', encoding='latin-1', low_memory=False, dtype=set_types)

#### Targets  
 - Set y(target) to proper category type
 - replace 'Not responsible' by 2

In [ ]:
df = df_train.copy(deep=True)
df['compliance'] = df['compliance'].fillna(2).replace({'0.0': 0, '1.0': 1}).astype('category')
np.bincount(df['compliance'])
df.groupby(['compliance'])['compliance'].count()

### Train set

#### Garbage Feature Collection
 - Clean garbage features

In [ ]:
garbage_features = ['city', 'clean_up_cost', 'disposition', 'grafitti_status', 'hearing_date', 'late_fee',
 'mailing_address_str_name', 'mailing_address_str_number', 'non_us_str_code', 'state_fee', 'ticket_id',
 'violation_code', 'violation_street_name', 'violation_street_number', 'violation_zip_code', 'violator_name',
 'payment_amount', 'balance_due']

df.drop(garbage_features, axis=1, inplace=True)

#### Feature engineering

In [ ]:
# Factorize in levels
df['zip_code'] = factorized(df['zip_code'])
df['agency_name'] = factorized(df['agency_name'])
df['inspector_name'] = factorized(df['inspector_name'])
df['violation_description'] = factorized(df['violation_description'])

# Detroit yes/no
df['state'] = factorized(df['state'])
df.loc[df['state'] != 0, 'state'] = 1

# USA yes/no
df.loc[df['country'] == 'USA', 'country'] = 1
df.loc[df['country'] != 1, 'country'] = 0

#### EDA impute NaN's

In [ ]:
null_columns = list(df.columns[df.isnull().any()])
null_objects = cat_features(df[null_columns])
null_numerics = num_features(df[null_columns])

# Convert date features to date and forward fill NaN's
df.loc[:, date_features(df)] = df.loc[:, date_features].apply(pd.to_datetime)
_ = [df.loc[:, x].fillna(method='ffill', inplace=True) for x in date_features(df)]

# Fill NaN's with mean for categorical/numeric columns
_ = [df.loc[:, x].fillna('', inplace=True) for x in null_objects]
_ = [df.loc[:, x].fillna(df.loc[:, x].mean(), inplace=True) for x in null_numerics]

assert df.isnull().sum().sum() == 0, 'Not all NaNs are removed'

#### Drop label '2' - NaN's

In [ ]:
# Split numerical features?????
not_nan = df['compliance'] != 2
X_train = df.loc[not_nan, num_features(df)]
y_train = df.loc[not_nan, 'compliance']

#### Train-validate split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

#### Scaling, transforming features

In [ ]:
# Scaling incl. class 2?????
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)

X_val = scaler.transform(X_val)

#### Train model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(learning_rate=0.05, max_depth=2, random_state=0).fit(X_train, y_train)

#### Train score

In [ ]:
y_train_pred = gb.predict(X_train)
sum(y_train_pred == y_train) / y_train.shape[0]

gb.score(X_train, y_train).round(4)

#### Validation score

In [ ]:
y_val_pred = gb.predict(X_val)
sum(y_val_pred == y_val) / y_val.shape[0]

gb.score(X_val, y_val).round(4)

### Test set

In [ ]:
df = df_test.copy(deep=True)

garbage_features = set(garbage_features) - set(['payment_amount', 'balance_due'])

# EDA Clean and split featureset
df.drop(garbage_features, axis=1, inplace=True)

#### Feature engineering

In [ ]:
# Factorize in levels
df['zip_code'] = factorized(df['zip_code'])
df['agency_name'] = factorized(df['agency_name'])
df['inspector_name'] = factorized(df['inspector_name'])
df['violation_description'] = factorized(df['violation_description'])

# Detroit yes/no
df['state'] = factorized(df['state'])
df.loc[df['state'] != 0, 'state'] = 1

# USA yes/no
df.loc[df['country'] == 'USA', 'country'] = 1
df.loc[df['country'] != 1, 'country'] = 0

#### EDA impute NaN's

In [ ]:
null_columns = list(df.columns[df.isnull().any()])
null_objects = cat_features(df[null_columns])
null_numerics = num_features(df[null_columns])

# Convert date features to date and forward fill NaN's
df.loc[:, date_features(df)] = df.loc[:, date_features].apply(pd.to_datetime)
_ = [df.loc[:, x].fillna(method='ffill', inplace=True) for x in date_features(df)]

# Fill NaN's with mean for categorical/numeric columns
_ = [df.loc[:, x].fillna('', inplace=True) for x in null_objects]
_ = [df.loc[:, x].fillna(df.loc[:, x].mean(), inplace=True) for x in null_numerics]

assert df.isnull().sum().sum() == 0, 'Not all NaNs are removed'

#### Scaling, transforming numerical features

In [ ]:
df_num_test = df.loc[:, num_features(df)]
X_test= scaler.transform(df_num_test)

#### Prediction

In [ ]:
y_pred = gb.predict(X_test)
pd.DataFrame(list(zip(y_pred, df_test['ticket_id']))).set_index(1).sample(5)

In [ ]:
np.bincount(y_pred)

#### Test score

In [ ]:
# test set doesnot have labels - no score available